# SOLID 원칙  

S : 단일 책임 원칙  
O : 개방/폐쇄의 원칙  
L : 리스코프 치환 원칙  
I : 인터페이스 분리 원칙  
D : 의존성 역전 원칙  

## 단일 책임 원칙(Single Responsibility Priciple - SRP)  

- 소프트웨어 컴포넌트(클래스)가 단 하나의 책임을 져야한다는 원칙.  
    --> 구체적인 일을 담당한다는 뜻으로, 변화해야 할 이유도 하나이다.  
- 필요한 일 이상의 것을 하거나 너무 많은 것을 알고 있는 객체는 서로 다른 행동을 그룹화한 것이므로, 유지보수가 어려워진다.  
- 클래스에 있는 프로퍼티와 속성이 항상 메서드를 통해서 사용되도록 하는 것! -> 관련된 개념이기 때문에 동일한 추상화로 묶는 것이 가능하다.  

### 너무 많은 책임을 가진 클래스  

밑의 예시는 SRP를 준수하지 않는 디자인이다.  
- 독립적인 동작을 하는 메서드를 하나의 인터페이스에 정의했다는 점.  ( 각각의 동작은 나머지 부분과 독립적으로 수행 가능 )  

In [1]:
class SystemMonitor:

    def load_activity(self):
        '''소스에서 처리할 이벤트를 가져오기'''

    def identify_events(self):
        '''가져온 데이터를 파싱하여 도메인 객체 이벤트로 변환'''

    def stream_events(self):
        '''파싱한 이벤트를 외부 에이전트로 전송'''



### 책임 분산  

모든 메서드를 다른 클래스로 분리하여 각 클래스마다 단일 책임을 갖게하면 된다.  
- 각자 책임을 가진 여러 객체로 만들고, 이들 객체들과 협력하여 동일한 기능을 수행하는 객체를 만들 수 있다.  
    --> 각각의 객체들은 특정한 기능을 캡슐화하여 나머지 객체들에 영향을 미치지 않으며, 명확하고 구체적인 의미를 가지게 된다.  
- 각 클래스의 유지보수가 쉽다.
    --> 다른 부분에서 다른 용도로 사용한다고 하면, 필요한 메서드만을 상속을 받기 때문에 이치에 맞게된다.  
* 각 클래스가 딱 하나의 메서드를 가져야 한다는 것을 뜻하는 것은 아님에 주의해야 하며, 처리해야 할 로직이 같은 경우 하나의 클래스에 여러 메서드를 추가할 수 있다.  



## 개방/폐쇄 원칙(Open/Close Principle)  

- 모듈이 개방되어 있으면서도 폐쇄되어야 한다는 원칙.  
- 클래스를 디자인할 때 유지보수가 쉽도록 로직을 캡슐화하여 확장에는 개방되고 수정에는 폐쇄되도록 해야 한다.  
    --> 확장 가능하고, 새로운 요구사항이나 도메인 변화에 잘 적응하는 코드를 작성해야 한다는 뜻!  
    --> 새로운 문제 발생 시, 새로운 것을 추가만 할 뿐 기존 코드는 그대로 유지해야 한다는 것.  

### 개방/폐쇄 원칙을 따르지 않는 경우 유지보수의 어려움  

밑의 코드는 개방/폐쇄 원칙을 따르지 않는 예제로 유지보수의 어려움과 비유연성을 확인해볼 수 있다.  

- 이벤트 유형을 결정하는 논리가 일체형으로 중앙 집중화되기 때문에, 이벤트가 늘어날수록 메서드의 크기가 커지므로 결국 매우 큰 메서드가 될 것이다.  
- 메서드가 수정을 위해 닫히지 않았으므로, 새로운 유형의 이벤트를 추가할 때마다 메서드를 수정해야 한다.  

--> 새로운 이벤트가 추가될 때 이미 존재하는 코드를 변경하지 않고 코드를 확장하여 새로운 유형의 이벤트를 지원해야 한다.(개방원칙).  

In [6]:
class Event:
    def __init__(self, raw_data):
        self.raw_data = raw_data


class UnknownEvent(Event):
    '''데이터만으로 식별할 수 없는 이벤트'''


class LoginEvent(Event):
    '''로그인 사용자에 의한 이벤트'''


class LogoutEvent(Event):
    '''로그아웃 사용자에 의한 이벤트'''


class SystemMonitor:
    '''시스템에서 발생한 이벤트 분류'''

    def __init__(self, event_data):
        self.event_data = event_data

    def identify_event(self):
        if self.event_data["before"]["session"] == 0 and self.event_data["after"]["session"] == 1:
            return LoginEvent(self.event_data)
        elif self.event_data["before"]["session"] == 1 and self.event_data["after"]["session"] == 0:
            return LogoutEvent(self.event_data)
        return UnknownEvent(self.event_data)

In [7]:
l1 = SystemMonitor({"before": {"session": 0}, "after": {"session": 1}})
l1.identify_event().__class__.__name__

'LoginEvent'

In [8]:
l1 = SystemMonitor({"before": {"session": 1}, "after": {"session": 0}})
l1.identify_event().__class__.__name__

'LogoutEvent'

In [9]:
l1 = SystemMonitor({"before": {"session": 1}, "after": {"session": 1}})
l1.identify_event().__class__.__name__

'UnknownEvent'

### 확장성을 가진 이벤트 시스템으로 리팩토링  

- 개방/폐쇄 원칙을 따르는 디자인을 달성하려면 추상화를 해야 한다!  
- 분류 메서드는 특정 이벤트 타입 대신에 일반적인 인터페이스를 따르는 제네릭 이벤트와 동작한다.  
- __subclasses__()를 통하여 이벤트 유형을 찾는다.  

In [11]:
class Event:
    def __init__(self, raw_data):
        self.raw_data = raw_data

    @staticmethod
    def meets_condition(event_data: dict):
        return False


class UnknownEvent(Event):
    '''데이터만으로 식별할 수 없는 이벤트'''


class LoginEvent(Event):
    @staticmethod
    def meets_condition(event_data: dict):
        return event_data["before"]["session"] == 0 and event_data["after"]["session"] == 1
    

class LogoutEvent(Event):
    @staticmethod
    def meets_condition(event_data: dict):
        return event_data["before"]["session"] == 1 and event_data["after"]["session"] == 0


class SystemMonitor:
    def __init__(self, event_data):
        self.event_data = event_data

    def identify_event(self):
        for event_cls in Event.__subclasses__():
            try:
                if event_cls.meets_condition(self.event_data):
                    return event_cls(self.event_data)
            except KeyError:
                continue
        return UnknownEvent(self.event_data)

### 이벤트 시스템 확장  

밑의 코드를 통해 원하는 대로 확장 가능하다는 것을 확인 할 수 있다.  

- 새 이벤트를 추가하였지만 SystemMonitor 클래스의 메서드는 수정하지 않았으므로, 새로운 유형의 이벤트에 대해서 폐쇄되어 있다고 할 수 있다.  
- 반대로 Event 클래스는 필요할 때마다 새로운 유형의 이벤트를 추가할 수 있게 해주므로 확장에 대해 개방되어 있다고 할 수 있다.

In [12]:
class TransactionEvent(Event):
    @staticmethod
    def meets_condition(event_data: dict):
        return event_data["after"].get("transaction") is not None

In [13]:
l1 = SystemMonitor({"before": {"session": 0}, "after": {"session": 1}})
l1.identify_event().__class__.__name__
l2 = SystemMonitor({"before": {"session": 1}, "after": {"session": 0}})
l2.identify_event().__class__.__name__
l3 = SystemMonitor({"before": {"session": 1}, "after": {"session": 1}})
l3.identify_event().__class__.__name__
l4 = SystemMonitor({"after": {"transaction": "Tx001"}})
l4.identify_event().__class__.__name__

'TransactionEvent'

### OCP 최종 정리  
- 다형성의 효과적인 사용과 밀접하게 연관되어 있다.  
- 코드를 변경하지 않고 기능을 확장하기 위해서는 보호하려는 추상화에 대해서 적절한 폐쇄를 해야 한다는 것!   